# Install Imports

In [1]:
pip install atproto -q
pip install pandas -q

SyntaxError: invalid syntax (2308973540.py, line 1)

# Create SQL Database

In [52]:
import multiprocessing
import signal
import time
from collections import defaultdict
from types import FrameType
from typing import Any

from atproto import CAR, AtUri, FirehoseSubscribeReposClient, firehose_models, models, parse_subscribe_repos_message


In [33]:
# Database setup (storing raw data instead of overloading memory)
conn = sqlite3.connect("firehose.db")
cur = conn.cursor()
cur.execute("""
    CREATE TABLE IF NOT EXISTS firehose (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        timestamp TEXT,
        repo TEXT,
        message TEXT
    )
""")
conn.commit()


## Start scraping data with Firehose and store in Database

In [35]:
from atproto import FirehoseSubscribeReposClient, parse_subscribe_repos_message
import sqlite3
import json

# Limit messages for testing (set to None for unlimited streaming)
MAX_MESSAGES = 5
message_count = 0


# Initialize Firehose client
client = FirehoseSubscribeReposClient()

def on_message_handler(message) -> None:
    global message_count
    if MAX_MESSAGES and message_count >= MAX_MESSAGES:
        print("Reached message limit. Stopping Firehose.")
        client.stop()
        return

    try:
        parsed_msg = parse_subscribe_repos_message(message)
        message_json = json.dumps(parsed_msg, default=str)

        # Store in SQLite instead of printing to prevent memory overload
        cur.execute("INSERT INTO firehose (timestamp, repo, message) VALUES (?, ?, ?)", 
                    (parsed_msg["time"], parsed_msg["repo"], message_json))
        conn.commit()

        print(f"Stored message {message_count + 1}: {parsed_msg['repo']}")
        message_count += 1

    except Exception as e:
        print(f"Error processing message: {e}")

# Clear the database before inserting new messages
cur.execute("DELETE FROM firehose")
conn.commit()

# Start Firehose listener
client.start(on_message_handler)


Stored message 1: did:plc:k7bmm7zmvi3l7yol726zjd5f
Stored message 2: did:plc:realist5mcrfawzhtlnqfa6q
Stored message 3: did:plc:uxi76mlhxwx6fwgp3wzgcpaj
Stored message 4: did:plc:hld7amk5yewgff6e3pnwztk3
Stored message 5: did:plc:7cjnuxacad65c47722mdnbyo
Reached message limit. Stopping Firehose.


## Example Output:


In [49]:
import sqlite3
import json

# Connect to the database
conn = sqlite3.connect("firehose.db")
cur = conn.cursor()

# Fetch 3 sample rows
cur.execute("SELECT timestamp, repo, message FROM firehose LIMIT 1")
rows = cur.fetchall()

# Display the results
for row in rows:
    timestamp, repo, message_json = row
    message = json.loads(message_json)  # Convert JSON string to dictionary
    print(f"\n📌 Entry from {repo} at {timestamp}:")
    print(json.dumps(message, indent=2))  # Pretty print JSON

conn.close()



📌 Entry from did:plc:k7bmm7zmvi3l7yol726zjd5f at 2025-02-21T14:15:49.415Z:
"blobs=[] blocks=b':\\xa2eroots\\x81\\xd8*X%\\x00\\x01q\\x12 :\\xef\\xec\\xd5\\xbf.\\xbd?\\xe0S\\x89`2\\xd6\\x91\\x96>IS\\xbff\\xb9\\xf2\\xfb\\xfdUH\\xbf\\xc3\\x17\\xc76gversion\\x01\\x84\\x08\\x01q\\x12 \\xcf\\x1a\\x0ck\\xba&\\x95Z\\xcc\\xb17P\\xf8\\xdf\\x8c9\\xab\\xd4\\xd8~a\\x17\\xd9a\\x82|Ry\\xa9\\xec\\rU\\xa2ae\\x89\\xa4akX app.bsky.feed.like/3lb5zq2rm3c2qap\\x00at\\xd8*X%\\x00\\x01q\\x12 \\xb1\\x1f \\x12\\x1a\\xfa\\xa6Q\\xb5!\\xb3\\xee\\x80C3m\\xca\"F\\x14\\xcc~\\xdd\\xb8,\\xa6S\\x84\\xc1z!xav\\xd8*X%\\x00\\x01q\\x12 \\x92\\x98\\x83\\xd81i\\x05n\\xe6\\xc9\\x0c\\xaf\\x1d\\x08\\xeb\\x81\\xc8/\\xf1\\x8d/\\xa2;\\xd6f\\xef\\xccx\\xe1\\xef_\\t\\xa4akJkakxfxlg22ap\\x16at\\xd8*X%\\x00\\x01q\\x12 \\xccG\\x85W\\x9e\\xc8\\x8f\\xf3\\x8d\\xa4\\xb6\\xf0\\xdft\\xcd\\x9f.\\x85\\x0fT\\x1a\\x8b\\xe1\\xc0}X\\t\\x99\\x8a\\x8cE\\xb8av\\xd8*X%\\x00\\x01q\\x12 \\x18\\xe0\\x12D\\x16\\xfa\\x99\\x1e#\\xf4\\x1d\\xef\\x82\\xba\\x14\

In [55]:
_INTERESTED_RECORDS = {
    models.ids.AppBskyFeedPost: models.AppBskyFeedPost,  # Regular posts
    models.ids.AppBskyFeedLike: models.AppBskyFeedLike,  # Likes
    models.ids.AppBskyFeedRepost: models.AppBskyFeedRepost,  # Reposts
    models.ids.AppBskyGraphFollow: models.AppBskyGraphFollow,  # Follows
    models.ids.AppBskyGraphBlock: models.AppBskyGraphBlock,  # Blocks
    models.ids.AppBskyFeedGenerator: models.AppBskyFeedGenerator,  # Custom feeds
    models.ids.AppBskyGraphList: models.AppBskyGraphList,  # Lists (e.g., user-created lists)
    models.ids.AppBskyGraphListitem: models.AppBskyGraphListitem,  # List items (adding/removing users)
    models.ids.AppBskyActorProfile: models.AppBskyActorProfile,  # Profile updates
    models.ids.AppBskyEmbedImages: models.AppBskyEmbedImages,  # Image attachments
    models.ids.AppBskyEmbedExternal: models.AppBskyEmbedExternal,  # External link previews
    models.ids.AppBskyEmbedRecord: models.AppBskyEmbedRecord,  # Embedded posts
    models.ids.AppBskyEmbedRecordWithMedia: models.AppBskyEmbedRecordWithMedia,  # Embedded posts + media
    models.ids.AppBskyLabelerService: models.AppBskyLabelerService,  # Moderation labels
    models.ids.ComAtprotoRepoStrongRef: models.ComAtprotoRepoStrongRef,  # References to repo content
}


In [56]:
    def worker_main(cursor_value: multiprocessing.Value, pool_queue: multiprocessing.Queue) -> None:
        signal.signal(signal.SIGINT, signal.SIG_IGN)

        while True:
            message = pool_queue.get()

            commit = parse_subscribe_repos_message(message)
            if not isinstance(commit, models.ComAtprotoSyncSubscribeRepos.Commit):
                continue

            if commit.seq % 20 == 0:
                cursor_value.value = commit.seq

            if not commit.blocks:
                continue

            ops = _get_ops_by_type(commit)

            # 🔹 Print ALL RECORDS, not just posts
            for record_type, actions in ops.items():
                for created_record in actions['created']:
                    author = created_record['author']
                    record = created_record['record']

                    inlined_text = getattr(record, 'text', '(No text)')  # Handle missing 'text' for likes/follows
                    print(f'NEW EVENT [TYPE={record_type}][CREATED_AT={record.created_at}][AUTHOR={author}]: {inlined_text}')


In [1]:
import multiprocessing
import signal
import time
from collections import defaultdict
from types import FrameType
from typing import Any

from atproto import CAR, AtUri, FirehoseSubscribeReposClient, firehose_models, models, parse_subscribe_repos_message

# Define all records of interest
_INTERESTED_RECORDS = {
    models.ids.AppBskyFeedPost: models.AppBskyFeedPost,  # Regular posts
    models.ids.AppBskyFeedLike: models.AppBskyFeedLike,  # Likes
    models.ids.AppBskyFeedRepost: models.AppBskyFeedRepost,  # Reposts
    models.ids.AppBskyGraphFollow: models.AppBskyGraphFollow,  # Follows
    models.ids.AppBskyGraphBlock: models.AppBskyGraphBlock,  # Blocks
    models.ids.AppBskyFeedGenerator: models.AppBskyFeedGenerator,  # Custom feeds
    models.ids.AppBskyGraphList: models.AppBskyGraphList,  # Lists
    models.ids.AppBskyGraphListitem: models.AppBskyGraphListitem,  # List items (adding/removing users)
    models.ids.AppBskyActorProfile: models.AppBskyActorProfile,  # Profile updates
    models.ids.AppBskyEmbedImages: models.AppBskyEmbedImages,  # Image attachments
    models.ids.AppBskyEmbedExternal: models.AppBskyEmbedExternal,  # External link previews
    models.ids.AppBskyEmbedRecord: models.AppBskyEmbedRecord,  # Embedded posts
    models.ids.AppBskyEmbedRecordWithMedia: models.AppBskyEmbedRecordWithMedia,  # Embedded posts + media
    models.ids.AppBskyLabelerService: models.AppBskyLabelerService,  # Moderation labels
    models.ids.ComAtprotoRepoStrongRef: models.ComAtprotoRepoStrongRef,  # References to repo content
}

# Limit the number of messages to process
MAX_EVENTS = 50
event_counter = multiprocessing.Value('i', 0)  # Shared counter


def _get_ops_by_type(commit: models.ComAtprotoSyncSubscribeRepos.Commit) -> defaultdict:
    """Extracts operations from the commit, categorizing them by type."""
    operation_by_type = defaultdict(lambda: {'created': [], 'deleted': []})
    car = CAR.from_bytes(commit.blocks)

    for op in commit.ops:
        uri = AtUri.from_str(f'at://{commit.repo}/{op.path}')

        if op.action == 'create':
            if not op.cid:
                continue

            create_info = {
                'uri': str(uri),
                'cid': str(op.cid),
                'author': commit.repo,
            }

            record_raw_data = car.blocks.get(op.cid)
            if not record_raw_data:
                continue

            record = models.get_or_create(record_raw_data, strict=False)
            record_type = _INTERESTED_RECORDS.get(uri.collection)

            if record_type and models.is_record_type(record, record_type):
                operation_by_type[uri.collection]['created'].append({'record': record, **create_info})

        elif op.action == 'delete':
            operation_by_type[uri.collection]['deleted'].append({'uri': str(uri)})

    return operation_by_type


def worker_main(cursor_value: multiprocessing.Value, pool_queue: multiprocessing.Queue, event_counter: multiprocessing.Value) -> None:
    """Worker function that processes Firehose messages."""
    signal.signal(signal.SIGINT, signal.SIG_IGN)

    while True:
        message = pool_queue.get()

        with event_counter.get_lock():
            if event_counter.value >= MAX_EVENTS:
                print("✅ Reached event limit. Stopping Firehose...")
                client.stop()
                return

        commit = parse_subscribe_repos_message(message)
        if not isinstance(commit, models.ComAtprotoSyncSubscribeRepos.Commit):
            continue

        if commit.seq % 20 == 0:
            cursor_value.value = commit.seq

        if not commit.blocks:
            continue

        ops = _get_ops_by_type(commit)

        # 🔹 Print ALL RECORD TYPES dynamically
        for record_type, actions in ops.items():
            for created_record in actions['created']:
                author = created_record['author']
                record = created_record['record']

                record_details = _format_record_details(record, record_type)

                print(f"🔹 NEW EVENT [TYPE={record_type}] [CREATED_AT={record.created_at}] [AUTHOR={author}] {record_details}")

                # Increase event counter
                with event_counter.get_lock():
                    event_counter.value += 1
                    if event_counter.value >= MAX_EVENTS:
                        print("✅ Event limit reached. Stopping...")
                        client.stop()
                        return


def _format_record_details(record, record_type):
    """Dynamically format details based on the type of record."""
    if record_type == models.ids.AppBskyFeedPost:
        return f"📜 POST: {record.text}"
    elif record_type == models.ids.AppBskyFeedLike:
        return f"❤️ LIKE → {record.subject}"
    elif record_type == models.ids.AppBskyFeedRepost:
        return f"🔁 REPOST → {record.subject}"
    elif record_type == models.ids.AppBskyGraphFollow:
        return f"➕ FOLLOWED: {record.subject}"
    elif record_type == models.ids.AppBskyGraphBlock:
        return f"🚫 BLOCKED: {record.subject}"
    elif record_type == models.ids.AppBskyActorProfile:
        return f"👤 PROFILE UPDATE: {record.displayName}"
    elif record_type in {models.ids.AppBskyEmbedImages, models.ids.AppBskyEmbedExternal}:
        return f"📷 MEDIA EMBEDDED"
    elif record_type == models.ids.ComAtprotoRepoStrongRef:
        return f"🔗 STRONG REF"
    else:
        return f"🧐 UNHANDLED TYPE"


def get_firehose_params(cursor_value: multiprocessing.Value) -> models.ComAtprotoSyncSubscribeRepos.Params:
    """Returns Firehose subscription parameters."""
    return models.ComAtprotoSyncSubscribeRepos.Params(cursor=cursor_value.value)


def measure_events_per_second(func: callable) -> callable:
    """Decorator to measure network event load."""
    def wrapper(*args) -> Any:
        wrapper.calls += 1
        cur_time = time.time()

        if cur_time - wrapper.start_time >= 1:
            print(f'📡 NETWORK LOAD: {wrapper.calls} events/second')
            wrapper.start_time = cur_time
            wrapper.calls = 0

        return func(*args)

    wrapper.calls = 0
    wrapper.start_time = time.time()
    return wrapper


def signal_handler(_: int, __: FrameType) -> None:
    """Handles keyboard interrupts gracefully."""
    print('🛑 Keyboard interrupt received. Waiting for queue to empty...')

    # Stop receiving new messages
    client.stop()

    # Drain the queue
    while not queue.empty():
        print('⏳ Waiting for queue to empty...')
        time.sleep(0.2)

    print('✅ Queue is empty. Terminating processes...')
    pool.terminate()
    pool.join()
    exit(0)


if __name__ == '__main__':
    signal.signal(signal.SIGINT, signal_handler)

    start_cursor = None

    params = None
    cursor = multiprocessing.Value('i', 0)
    if start_cursor is not None:
        cursor = multiprocessing.Value('i', start_cursor)
        params = get_firehose_params(cursor)

    client = FirehoseSubscribeReposClient(params)

    workers_count = multiprocessing.cpu_count() * 2 - 1
    max_queue_size = 10000

    queue = multiprocessing.Queue(maxsize=max_queue_size)
    pool = multiprocessing.Pool(workers_count, worker_main, (cursor, queue, event_counter))

    @measure_events_per_second
    def on_message_handler(message: firehose_models.MessageFrame) -> None:
        if cursor.value:
            client.update_params(get_firehose_params(cursor))

        queue.put(message)

    client.start(on_message_handler)


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'worker_main' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = r

📡 NETWORK LOAD: 4 events/second


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'worker_main' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = r

📡 NETWORK LOAD: 339 events/second


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'worker_main' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = r

...

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'worker_main' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = r

📡 NETWORK LOAD: 653 events/second


  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'worker_main' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'wor

📡 NETWORK LOAD: 683 events/second


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'worker_main' on <module '__main__' (built-in)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/Library/Developer/CommandLineTools/Library/Frameworks/Python3.framework/Versions/3.9/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = r